In [1]:
import os
import pandas as pd
import requests
import json
import requests
import multiprocessing as mp
from random import randint
from time import sleep

import sys
sys.path.insert(0,'/global/homes/b/bpb/repos/metatlas')

# sys.path.insert(0,'/global/project/projectdirs/metatlas/anaconda/lib/python2.7/site-packages' )
from rdkit import Chem
from metatlas import metatlas_objects as metob

('NERSC=', True)


In [2]:
# df = pd.read_pickle('/project/projectdirs/openmsi/projects/ben_run_pactolus/unique_compounds.pkl')
df = pd.read_csv('/project/projectdirs/metatlas/projects/unique_compounds.csv.gz')
df.rename(columns = {'monoisotopoic_mw':'monoisotopic_mw'},inplace=True)

/global/common/software/m2650/python-cori/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,2,5,6,7,10,11,12,13,14,15,25,26,28) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
all_compounds = metob.retrieve('Compounds',username = '*')

all_compounds_df = metob.to_dataframe(all_compounds)


all_compounds_df.keys()

In [ ]:
df = pd.read_pickle('/project/projectdirs/openmsi/projects/ben_run_pactolus/unique_compounds.pkl')
# df = pd.read_csv('/project/projectdirs/openmsi/projects/compound_data/jgi_molecules/new_jgi_compounds.csv')
df.rename(columns = {'monoisotopoic_mw':'monoisotopic_mw'},inplace=True)

In [6]:
df = df.convert_objects(convert_numeric=True)
df.keys()

/global/common/software/m2650/python-cori/lib/python2.7/site-packages/IPython/kernel/__main__.py:1: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  if __name__ == '__main__':


Index([u'Unnamed: 0', u'chebi_id', u'chebi_url', u'description', u'formula',
       u'hmdb_id', u'hmdb_url', u'img_abc_id', u'inchi', u'inchi_key',
       u'iupac_name', u'kegg_id', u'kegg_url', u'lipidmaps_id',
       u'lipidmaps_url', u'metacyc_id', u'mono_isotopic_molecular_weight',
       u'name', u'neutralized_2d_inchi', u'neutralized_2d_inchi_key',
       u'neutralized_inchi', u'neutralized_inchi_key', u'num_free_radicals',
       u'number_components', u'permanent_charge', u'pubchem_compound_id',
       u'pubchem_url', u'source', u'synonyms', u'wikipedia_url'],
      dtype='object')

In [9]:
df.sort_values('mono_isotopic_molecular_weight',inplace=True)
df.head(20).mono_isotopic_molecular_weight.tolist()

[1.007825032,
 1.00837361191,
 2.015650064,
 2.015650064,
 4.028203556,
 6.032098556,
 7.01545597009,
 7.01600455,
 8.023829582000001,
 9.01108504018,
 9.012182199999998,
 13.024955464000001,
 14.0047197397,
 14.015650064,
 14.016198643900001,
 15.023475095999999,
 15.024023675899999,
 15.041154107899999,
 15.99491462,
 15.9960117798]

In [10]:
df.head(100)

,Unnamed: 0,chebi_id,chebi_url,description,formula,hmdb_id,hmdb_url,img_abc_id,inchi,inchi_key,...,neutralized_inchi,neutralized_inchi_key,num_free_radicals,number_components,permanent_charge,pubchem_compound_id,pubchem_url,source,synonyms,wikipedia_url
52361,151834,NaN,NaN,NaN,H,NaN,NaN,NaN,InChI=1S/H,YZCKVEUIGOORGS-UHFFFAOYSA-N,...,InChI=1S/H,YZCKVEUIGOORGS-UHFFFAOYSA-N,1,1,0,5362549.0,http://pubchem.ncbi.nlm.nih.gov/compound/5362549,wikidata,hydrogen(.)///Hydrogen atom///monohydrogen///H...,NaN
113147,62173,NaN,NaN,NaN,H-,NaN,NaN,NaN,InChI=1S/H/q-1,KLGZELKXQMTEMM-UHFFFAOYSA-N,...,InChI=1S/H/q-1,KLGZELKXQMTEMM-UHFFFAOYSA-N,0,1,-1,166653.0,http://pubchem.ncbi.nlm.nih.gov/compound/166653,wikidata,Hydride///hydrogen anion///AC1L4YHB///CHEBI:29...,NaN
18168,120562,CHEBI:29299,http://www.ebi.ac.uk/chebi/searchId.do?chebiId...,NaN,H2,NaN,NaN,NaN,InChI=1S/H2/h1H/i1+0H,UFHFLCQGNIYNRP-HXFQMGJMSA-N,...,InChI=1S/H2/h1H/i1+0H,UFHFLCQGNIYNRP-HXFQMGJMSA-N,0,1,0,6914304.0,http://pubchem.ncbi.nlm.nih.gov/compound/6914304,chebi,diprotium///CHEBI:29299///(1)H2,NaN
11893,120578,CHEBI:18276,http://www.ebi.ac.uk/chebi/searchId.do?chebiId...,An elemental molecule consisting of two hydrog...,H2,HMDB01362,http://www.hmdb.ca/metabolites/HMDB01362,NaN,InChI=1S/H2/h1H,UFHFLCQGNIYNRP-UHFFFAOYSA-N,...,InChI=1S/H2/h1H,UFHFLCQGNIYNRP-UHFFFAOYSA-N,0,1,0,783.0,http://pubchem.ncbi.nlm.nih.gov/compound/783,chebi///hmdb,Hydrogen///Dihydrogen///Molecular hydrogen///1...,NaN
106035,120571,CHEBI:29294,http://www.ebi.ac.uk/chebi/searchId.do?chebiId...,NaN,H2,NaN,NaN,NaN,InChI=1S/H2/h1H/i1+1D,UFHFLCQGNIYNRP-VVKOMZTBSA-N,...,InChI=1S/H2/h1H/i1+1D,UFHFLCQGNIYNRP-VVKOMZTBSA-N,0,1,0,24523.0,http://pubchem.ncbi.nlm.nih.gov/compound/24523,chebi///wikidata,DEUTERIUM///Dideuterium///7782-39-0///Heavy hy...,NaN
105217,120556,CHEBI:29298,http://www.ebi.ac.uk/chebi/searchId.do?chebiId...,NaN,H2,NaN,NaN,NaN,InChI=1S/H2/h1H/i1+2T,UFHFLCQGNIYNRP-JMRXTUGHSA-N,...,InChI=1S/H2/h1H/i1+2T,UFHFLCQGNIYNRP-JMRXTUGHSA-N,0,1,0,24824.0,http://pubchem.ncbi.nlm.nih.gov/compound/24824,chebi,TRITIUM///Hydrogen-3///10028-17-8///ditritium/...,NaN
101611,39731,CHEBI:33979,http://www.ebi.ac.uk/chebi/searchId.do?chebiId...,NaN,Li+,NaN,NaN,NaN,InChI=1S/Li/q+1,HBBGRARXTFLTSG-UHFFFAOYSA-N,...,InChI=1S/Li/q+1,HBBGRARXTFLTSG-UHFFFAOYSA-N,0,1,1,28486.0,http://pubchem.ncbi.nlm.nih.gov/compound/28486,chebi///wikidata,lithium ion///Lithium(1+)///Lithium element///...,NaN
100424,134107,NaN,NaN,NaN,Li,NaN,NaN,NaN,InChI=1S/Li,WHXSMMKQMYFTQS-UHFFFAOYSA-N,...,InChI=1S/Li,WHXSMMKQMYFTQS-UHFFFAOYSA-N,1,1,0,3028194.0,http://pubchem.ncbi.nlm.nih.gov/compound/3028194,wikidata,"LITHIUM///7439-93-2///litio///Lithium, metalli...",NaN
161286,113353,CHEBI:125696,http://www.ebi.ac.uk/chebi/searchId.do?chebiId...,NaN,HLi,NaN,NaN,NaN,InChI=1S/Li.H,SIAPCJWMELPYOE-UHFFFAOYSA-N,...,InChI=1S/Li.H,SIAPCJWMELPYOE-UHFFFAOYSA-N,2,1,0,NaN,NaN,chebi,NaN,NaN
128122,97432,CHEBI:35102,http://www.ebi.ac.uk/chebi/searchId.do?chebiId...,NaN,Be+2,NaN,NaN,NaN,InChI=1S/Be/q+2,PWOSZCQLSAMRQW-UHFFFAOYSA-N,...,InChI=1S/Be/q+2,PWOSZCQLSAMRQW-UHFFFAOYSA-N,0,1,2,107649.0,http://pubchem.ncbi.nlm.nih.gov/compound/107649,chebi///wikidata,beryllium(2+)///Beryllium atom///Beryllium ele...,NaN


In [11]:
inchis = df[(~ pd.isnull(df.inchi_key))].inchi_key

In [ ]:
baumin_inchi = df[(df.metatlas_inchi_key.str.contains('JZRMMLYGOBWIGY')) & (~ pd.isnull(df.metatlas_inchi))].metatlas_inchi
with open('/global/homes/b/bpb/inchi/baumin_inchi.txt','w') as fid:
    for my_inchi in baumin_inchi:
        m = Chem.MolFromInchi(my_inchi)
        if m:
            fid.write('%s\n'%my_inchi)
baumin_inchi

In [ ]:
%system ls /project/projectdirs/openmsi/projects/ben_trees/*.npy

In [ ]:
import numpy as np
d = np.load('/project/projectdirs/openmsi/projects/ben_trees/metacyc_max_depth_5.npy')

In [ ]:
# mw = [dd[-1] for dd in d if dd[-1] > 0]
from matplotlib import pyplot as plt
%matplotlib inline
out = plt.hist(df.monoisotopic_mw,bins = 100,range=(0,1200))
plt.xlabel('molecular weight')
plt.ylabel('# structures')
plt.show()

In [12]:
chunks = [inchis[x:x+1000] for x in xrange(0, len(inchis), 1000)]


In [ ]:
# inchis = df[(~ pd.isnull(df.metatlas_inchi)) & 
#                  (df.monoisotopic_mw > 100) &
#                 (df.formula.str.contains('C\d+')) &
#                (~ df.formula.str.contains('F')) &
#            (~ df.formula.str.contains('Al')) &
#            (~ df.formula.str.contains('Cl')) &
#            (~ df.formula.str.contains('Si')) &
#            (~ df.formula.str.contains('B')) &
#            (~ df.formula.str.contains('Br')) &
#            (~ df.formula.str.contains('Se'))].metatlas_inchi
# # for i in inchis[:1000]:
# #     print i

In [ ]:
inchis[:10]

In [13]:
%system mkdir /project/projectdirs/metatlas/projects/pubchem_info

[]

In [ ]:
with open('/project/projectdirs/metatlas/projects/pubchem_info/inchis_all.txt','w') as fid:
    for my_inchi in inchis:
        m = Chem.MolFromInchi(my_inchi)
        if m:
            fid.write('%s\n'%my_inchi)

In [ ]:
for i,inchi_list in enumerate(chunks):
    with open('/global/homes/b/bpb/inchi/inchis_%d.txt'%i,'w') as fid:
        for my_inchi in inchi_list:
            m = Chem.MolFromInchi(my_inchi)
            if m:
                fid.write('%s\n'%my_inchi)

In [ ]:
for i,inchi_list in enumerate(chunks):
    with open('/global/homes/b/bpb/inchi/inchis_%d.txt'%i,'w') as fid:
        for my_inchi in inchi_list:
            m = Chem.MolFromInchi(my_inchi)
            if m:
                fid.write('%s\n'%my_inchi)


In [ ]:
inchi_keys = df[(~ pd.isnull(df.metatlas_inchi)) & 
                 (df.monoisotopic_mw > 100) &
                (df.formula.str.contains('C\d+'))].metatlas_inchi.unique()

In [ ]:
out_dir = '/project/projectdirs/openmsi/projects/ben_run_pactolus/inchi_key_to_pubchem_names'
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

# remove those that have already been done

In [ ]:
import glob
files = glob.glob(os.path.join(out_dir,'*.json'))
done_inchi_key = []
for f in files:
    done_inchi_key.append(os.path.basename(f).split('.')[0])
inchi_keys = list(set(inchi_keys) - set(done_inchi_key))

In [ ]:
len(inchi_keys)

In [ ]:
def write_pubchem_info_to_file(inchi_key):
    suffix = '.json'
    fname = os.path.join(out_dir, inchi_key + suffix)
    if not os.path.isfile(fname):
        url = 'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/inchikey/%s/synonyms/json'%inchi_key
        response = requests.get(url)
        try: 
            with open(fname, 'w') as fid:
                json.dump(response.json(), fid)
        except:
            print "could not query", inchi_key

In [ ]:
for ik in inchi_keys[1:]:
    print ik
    write_pubchem_info_to_file(ik)

In [ ]:
# pool = mp.Pool(processes=20)
# pool.map(write_pubchem_info_to_file, inchi_keys[1:100])

In [ ]:
url = 'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/inchikey/%s/synonyms/json'%'CKLJMWTZIZZHCS-REOHCLBHSA-N'
response = requests.get(url)
response.json()

In [ ]:
response.json()

In [ ]:
url = 'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/%s/json'%'Baumin'
response = requests.get(url)
response.json()